In [1]:
sc

StatementMeta(SmallPool, 26, 2, Finished, Available, Finished)

<SparkContext master=yarn appName=Transformation5_Window_function_SmallPool_1727186900>

Run the 'mount_confuguration' notebook to access the mount

In [2]:
mssparkutils.notebook.run('mount_configuration')

StatementMeta(SmallPool, 26, 3, Finished, Available, Finished)

'Mounting'

Store the mount into a variable

In [3]:
job_id = mssparkutils.env.getJobId()
mount_point = 'synfs:/notebook/'+job_id+'/lake'

StatementMeta(SmallPool, 26, 4, Finished, Available, Finished)

Read the source file into a dataframe

In [4]:
df = spark.read.parquet(mount_point+'/DataSorted/*.parquet')

StatementMeta(SmallPool, 26, 5, Finished, Available, Finished)

In [5]:
display(df)

StatementMeta(SmallPool, 26, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 72de39e5-28c7-49e0-96f5-076f72810437)

The variable window stores a window specification that defines how rows should be ordered by UnEmployed_Rate_Percentage for use in window functions.

In [10]:
from pyspark.sql.window import Window

window = Window.orderBy('UnEmployed_Rate_Percentage')

StatementMeta(SmallPool, 26, 11, Finished, Available, Finished)

In [11]:
print(window)

StatementMeta(SmallPool, 26, 12, Finished, Available, Finished)

Creating row_number column using the window function

In [13]:
from pyspark.sql.functions import row_number,rank,dense_rank,min,max
df_row = df.withColumn('Row_Number',row_number().over(window))

StatementMeta(SmallPool, 26, 14, Finished, Available, Finished)

In [14]:
display(df_row)

StatementMeta(SmallPool, 26, 15, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 919cca75-5500-4c12-ae68-11114ecd73e3)

In [16]:
print(df_row.count())
df_row.agg(min('Row_Number')).show()
df_row.agg(max('Row_Number')).show()

StatementMeta(SmallPool, 26, 17, Finished, Available, Finished)

1477
+---------------+
|min(Row_Number)|
+---------------+
|              1|
+---------------+

+---------------+
|max(Row_Number)|
+---------------+
|           1477|
+---------------+



Creating rank column using the window function

In [29]:
df_rank = df_row.withColumn('rank',rank().over(window))

StatementMeta(SmallPool, 26, 30, Finished, Available, Finished)

In [30]:
display(df_rank)

StatementMeta(SmallPool, 26, 31, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, dd11cee0-8d3c-4247-a892-a0e9a8dad8cf)

Creating dense_rank column using the window function

In [46]:
df_denserank = df_rank.withColumn('Dense_Rank',dense_rank().over(window))

StatementMeta(SmallPool, 26, 47, Finished, Available, Finished)

Dropping the unwanted rows

In [47]:
df_final = df_denserank.drop('Row_Number','Rank')

StatementMeta(SmallPool, 26, 48, Finished, Available, Finished)

In [48]:
display(df_final)

StatementMeta(SmallPool, 26, 49, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 1f2f5d4d-cc96-424c-9f91-e4635db98dbb)

Creating a temporary view to query the data using sql

In [42]:
df_denserank.createOrReplaceTempView('ranks')

StatementMeta(SmallPool, 26, 43, Finished, Available, Finished)

dense_rank is 1 for the least value in UnEmployed rate percentage

In [43]:
%%sql
select * from ranks where dense_rank = 1

StatementMeta(SmallPool, 26, 44, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 19 fields>

dense_rank will be the maximum for the highest value in UnEmployed rate percentage

In [45]:
%%sql
select * from ranks where dense_rank = (select max(dense_rank) from ranks)

StatementMeta(SmallPool, 26, 46, Finished, Available, Finished)

<Spark SQL result set with 2 rows and 19 fields>

Writing the final data into a file

In [49]:
df_final.write.format('parquet')\
                  .mode('overwrite')\
                  .save(mount_point+'/WindowFunctions/')

StatementMeta(SmallPool, 26, 50, Finished, Available, Finished)